# Title

### Wormhole


### Terra Bridge




### Objective


### Data


### Query
The query used to retrieve the relevant data can be found [here](https://app.flipsidecrypto.com/velocity/queries/5027ee2e-4de7-41dd-8686-072046b3ce8b). Some main points to highlight:
* 
* 
* 

In [89]:
tokens = {
    'terra14tl83xcwqjy0ken9peu4pjjuu755lrry2uy25r': {'symbol':'whWETH',
                                                    'asset_chain':2},
    'terra1ce06wkrdm4vl6t0hvc0g86rsy27pu8yadg3dva': {'symbol':'whUSDT',
                                                    'asset_chain':2},
    'terra1e6mq63y64zcxz8xyu5van4tgkhemj3r86yvgu4':{'symbol':'wh',
                                                    'asset_chain':1},
       'terra190tqwgqx7s8qrknz6kckct7v607cu068gfujpk':{'symbol':'whSOL',
                                                    'asset_chain':1},
       'terra1hd9n65snaluvf7en0p4hqzse9eqecejz2k8rl5':{'symbol':'wh',
                                                    'asset_chain':1},
       'terra19phjr9363x44yukx6dl6d4zvn2wqc8gpkk0tm7':{'symbol':'whETH',
                                                    'asset_chain':1},
       'terra1tuleqeucunvs9rga3dq5h35ew3ml8uv9a4snzx':{'symbol':'whMIR',
                                                    'asset_chain':4},
       'terra19t8cxscre0jss7nejqy29c97fkg7els3v8zujg':{'symbol':'wh',
                                                    'asset_chain':1},
       'terra1kcthelkax4j9x8d3ny6sdag0qmxxynl3qtcrpy':{'symbol':'MINE',
                                                    'asset_chain':3},
       'terra16q5pke2vueu23y8punvj70h0cp0s0rc7vrzl57':{'symbol':'wh',
                                                    'asset_chain':1},
       'terra15gwkyepfc6xgca5t5zefzwy42uts8l2m4g40k6':{'symbol':'MIR',
                                                    'asset_chain':3},
       'terra14z56l0fp2lsf86zy3hty2z47ezkhnthtr9yq76':{'symbol':'ANC',
                                                    'asset_chain':3},
       'terra1k7tstwhf25wm45x329ny84xe4n2z2nnq9ekuv2':{'symbol':'wh',
                                                    'asset_chain':1},
       'terra1gdapfuda0dxtjc98raemhszntcxty7chyr0wpd':{'symbol':'whORION',
                                                    'asset_chain':1},
       'terra1aa7upykmmqqc63l924l5qfap8mrmx5rfdm0v55':{'symbol':'whWBTC',
                                                    'asset_chain':2},
       'terra1mddcdx0ujx89f38gu7zspk2r2ffdl5enyz2u03':{'symbol':'ORION',
                                                    'asset_chain':3},
       'terra1cetg5wruw2wsdjp7j46rj44xdel00z006e9yg8':{'symbol':'whWBNB',
                                                    'asset_chain':4},
       'terra1xu3nyee55dfe5vxqtg9m4rkex8u2t6n2etl08h':{'symbol':'whUSDT',
                                                    'asset_chain':5},
       'terra127ww0al7uj9vyg7cv2gtcqenhrjzlnq5vaq6dh':{'symbol':'whUST',
                                                    'asset_chain':5},
       'terra1fme0uqgjucph3kelyejm6yuj60e4te8647gx6v':{'symbol':'whLUNA',
                                                    'asset_chain':5},
       'terra1vlqeghv5mt5udh96kt5zxlh2wkh8q4kewkr0dd':{'symbol':'whUSDT',
                                                    'asset_chain':4},
       'terra1kkyyh7vganlpkj0gkc2rfmhy858ma4rtwywe3x':{'symbol':'whUSDC',
                                                    'asset_chain':5},
       'terra132zppkzw88j35hpwxkrpzymujekky3wcuru65n':{'symbol':'whATOM',
                                                    'asset_chain':4}
}

### Chain IDs:
* 1 -> Solana
* 2 -> Ethereum
* 3 -> Terra
* 4 -> BSC
* 5 -> Polygon

In [57]:
chain_ids = ['Solana', 'Ethereum', 'Terra', 'BSC', 'Polygon']

## Analysis

**Imports**

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (15.0, 5.0)
pd.set_option('display.max_columns', 120)
pd.options.display.max_colwidth = 10000

In [4]:
url_wh_receive = 'https://api.flipsidecrypto.com/api/v2/queries/657279fb-8a07-4089-bf08-307fb7f1325d/data/latest'
url_wh_send = 'https://api.flipsidecrypto.com/api/v2/queries/6cecbc05-4540-4c33-9557-585aec1af791/data/latest'

In [52]:
wh_receive_raw = pd.read_json(url_wh_receive)
wh_send_raw = pd.read_json(url_wh_send)

In [15]:
wh_receive = wh_receive_raw[['AMOUNT', 'BLOCK_TIMESTAMP', 'CHAIN_ID',
       'DENOM', 'ORIGINAL_SENDER', 'RECIPIENT', 'SENDER',
       'TX_ID', 'TX_MODULE', 'TX_STATUS', 'TX_TYPE']]
wh_receive = wh_receive[wh_receive.TX_STATUS == 'SUCCEEDED']
wh_receive.columns = [s.lower() for s in wh_receive.columns]

In [93]:
wh_send_raw = wh_send_raw[wh_send_raw.TX_STATUS == 'SUCCEEDED']
wh_send = wh_send_raw[['BLOCK_TIMESTAMP',
       'MSG_VALUE:EXECUTE_MSG:INITIATE_TRANSFER:ASSET:AMOUNT',
       'MSG_VALUE:EXECUTE_MSG:INITIATE_TRANSFER:ASSET:INFO:NATIVE_TOKEN:DENOM',
       'MSG_VALUE:EXECUTE_MSG:INITIATE_TRANSFER:ASSET:INFO:TOKEN:CONTRACT_ADDR',
       'MSG_VALUE:EXECUTE_MSG:INITIATE_TRANSFER:RECIPIENT_CHAIN',
       'MSG_VALUE:SENDER', 'TX_ID']]
wh_send.columns = ['block_timestamp',
       'amount',
       'denom',
       'token_contract_addr',
       'recipient_chain',
       'sender', 'tx_id']
wh_send.token_contract_addr=wh_send.token_contract_addr.apply(lambda x: x.replace("\"","") if x else '')
wh_send.denom=wh_send.denom.apply(lambda x: x.replace("\"","") if x else '')
wh_send.amount=wh_send.amount.apply(lambda x: x.replace("\"","") if x else '')
wh_send.sender=wh_send.sender.apply(lambda x: x.replace("\"","") if x else '')
wh_send['token_addr_name'] = wh_send.token_contract_addr.apply(lambda x: 
                                            f"{tokens[x]['symbol']}_{tokens[x]['asset_chain']}" if x in tokens else '')

<ipython-input-93-7a287166e421>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wh_send['token_addr_name'] = wh_send.token_contract_addr.apply(lambda x:


In [94]:
wh_send[wh_send.token_addr_name != ''].token_addr_name.value_counts()

whWETH_2     69
whSOL_1      41
wh_1         39
whORION_1    21
whETH_1      11
ORION_3       8
MIR_3         6
whUSDT_2      6
whMIR_4       4
ANC_3         3
whUST_5       3
whWBNB_4      1
whUSDT_4      1
whUSDT_5      1
whLUNA_5      1
whWBTC_2      1
whATOM_4      1
MINE_3        1
whUSDC_5      1
Name: token_addr_name, dtype: int64

In [95]:
wh_send.recipient_chain.apply(lambda n: chain_ids[n-1]).value_counts()

Solana      2425
BSC          321
Ethereum     121
Polygon       24
Name: recipient_chain, dtype: int64

In [96]:
wh_send.columns

Index(['block_timestamp', 'amount', 'denom', 'token_contract_addr',
       'recipient_chain', 'sender', 'tx_id', 'token_addr_name'],
      dtype='object')

In [97]:
wh_send['token_name'] = wh_send.apply(lambda r: r.denom if r.denom else r.token_addr_name, axis=1)

<ipython-input-97-98d1215aa1be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wh_send['token_name'] = wh_send.apply(lambda r: r.denom if r.denom else r.token_addr_name, axis=1)


In [98]:
wh_send.token_name.value_counts()

uusd         2328
uluna         344
whWETH_2       69
whSOL_1        41
wh_1           39
whORION_1      21
whETH_1        11
ORION_3         8
whUSDT_2        6
MIR_3           6
whMIR_4         4
ANC_3           3
whUST_5         3
whLUNA_5        1
MINE_3          1
whATOM_4        1
whUSDT_5        1
whWBTC_2        1
whUSDC_5        1
whUSDT_4        1
whWBNB_4        1
Name: token_name, dtype: int64